In [1]:
!pip install pycaret
!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


<h3>1. Introduction </h3>
This template notebook will give users (data scientists and data engineers) the opportunities to train and deploy regression machine learning models with ease, without having to write all of the code from scratch. <br> 
There are a few requirements for the user anyway, such as;
<li>the data location</li> 
<li>the target (dependent) variable in your dataset and</li>
<li>what kind of machine learning algorithm you will be performing. </li>

All of these values will be entered in terraform and will be automatically applied in the notebook. 

<h3>2. Imports</h3>

The libraries that are required for this model notebook are imported below 

In [1]:
import boto3, os, tarfile
from sagemaker import get_execution_role
from dotenv import load_dotenv
from load_data import load_data

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/lanre.bakare/Library/Application Support/sagemaker/config.yaml


<h3>3. Loading Data</h3>
Here the user is required to specify the location of the data that they will like to use for prediction. An helper function is used to load the data from S3. 

<em>Note: Your data needs to be an s3 bucket.</em>

In [2]:
# Variables Setup Stage
load_dotenv(".env")
role = get_execution_role()

# Env variables
data_location_s3 = os.getenv("data_location_s3")
algorithm_choice = os.getenv("algorithm_choice")
target = os.getenv("target")
endpoint_name = os.getenv("endpoint_name")
model_name = os.getenv("model_name")
data_location = 's3://{}'.format(data_location_s3)
print(data_location_s3, algorithm_choice, target, endpoint_name, model_name, data_location)

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/lanre.bakare/Library/Application Support/sagemaker/config.yaml
streaming-data-platform-ml-data/bakerloo.csv regression Bakerloo10 lanre-trial-endpoint lanre-test s3://streaming-data-platform-ml-data/bakerloo.csv


<h3>4. Read and display a sample of data</h3>

In [10]:
# Data Option 1: Use Data from S3
data = load_data(data_location)
df = data.copy()
df.head() 

,Bakerloo,Bakerloo10,dayOfWeek,hour,minute
0,23,24.0,1,13,38
1,23,22.0,1,13,40
2,23,24.0,1,13,41
3,23,24.0,1,13,42
4,23,24.0,1,13,43


## Importing Pycaret

<h3>5. Data Exploration</h3>

In [4]:

# Randomly shuffle the DataFrame
df_shuffled = df.sample(frac=1).reset_index(drop=True)

# Shuffling the dataset and splitting into train and test dataset
train_size = int(0.8 * len(df))
train_data = df_shuffled[:train_size]
test_data = df_shuffled[train_size:]

In [3]:
import importlib

# Import Pycaret Module
pycaret = importlib.import_module(f"pycaret.{algorithm_choice}")

In [5]:
# Set Up
pycaret.setup(data=train_data, target=target, session_id=123)

,Description,Value
0,Session id,123
1,Target,Bakerloo10
2,Target type,Regression
3,Original data shape,"(5701, 5)"
4,Transformed data shape,"(5701, 5)"
5,Transformed train set shape,"(3990, 5)"
6,Transformed test set shape,"(1711, 5)"
7,Numeric features,4
8,Preprocess,True
9,Imputation type,simple


<h3>6. Feature Engineering and Model Training</h3>

Here we are using the pycaret automl tool to train the model. The automl tool tries a number of machine learning algorithms depending on the type of machine learning problem you are trying to solve <br>
(regression, classification or time series). The automl tool then selects the best model based on the accuracy metrics of the tried models. 

In [7]:
# Option 1: let pycaret choose the best model for you
bestModel = pycaret.compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.7156,1.1151,1.0493,0.9800,0.0696,0.0511,0.1560
xgboost,Extreme Gradient Boosting,0.7168,1.2249,1.1007,0.9780,0.0704,0.0482,0.0120
lightgbm,Light Gradient Boosting Machine,0.7418,1.2384,1.1046,0.9779,0.0792,0.0546,0.3810
rf,Random Forest Regressor,0.6697,1.3131,1.1320,0.9764,0.0682,0.0402,0.0600
et,Extra Trees Regressor,0.6358,1.3532,1.1523,0.9757,0.0677,0.0377,0.0460
gbr,Gradient Boosting Regressor,0.8952,1.6712,1.2876,0.9702,0.0926,0.0685,0.0240
knn,K Neighbors Regressor,0.9180,1.8918,1.3678,0.9663,0.0936,0.0647,0.0070
dt,Decision Tree Regressor,0.7462,2.1759,1.4596,0.9608,0.0890,0.0422,0.0050
lar,Least Angle Regression,1.1629,2.7473,1.6545,0.9508,0.1358,0.0973,0.0040
lr,Linear Regression,1.1629,2.7473,1.6545,0.9508,0.1358,0.0973,0.2850


<h3>7. Model Evaluation: </h3>

Here we evaluate the performance of the best model, getting some visual representation of hyperparameters, features and other important details about the selected model.


In [8]:
# Evaluate model: View Hyperparameters, Confusion Matrix, Class Report, etc.
pycaret.evaluate_model(bestModel)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

<h3>8. Saving Model for future predition</h3>

Here we are using a function that saves the model to s3. 

In [9]:
# Save model
final_model = pycaret.finalize_model(bestModel)
pycaret.save_model(final_model, 'final_best_model')

Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['Bakerloo', 'dayOfWeek', 'hour',
                                             'minute'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=[],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('actual_estimator',
                 <catboost.core.CatBoostRegressor object at 0x29ddcb310>)])
Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['Bakerloo', 'dayOfWeek', 'hour',
                                              'minute'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('actual_estimator',
                  <catboost.core.CatBoostRegressor object at 0x29ddcb310>)]),
 'final_best_model.pkl')

In [ ]:
# Convert model into tar file and upload to S3

with tarfile.open('final_best_model.tar.gz', 'w:gz') as tar:
    tar.add('final_best_model.pkl')


s3 = boto3.client('s3')
s3.upload_file('final_best_model.tar.gz', model_name, 'final_best_model.tar.gz')

<h3>9. Deploying the model endpoints</h3> 

Here we use a function that creates the model endpoint in sagemaker. 

In [ ]:
# Deploy Model
from sagemaker.model import Model
model_data = f's3://{model_name}/final_best_model.tar.gz'
# Move model to s3. Member must satisfy regular expression pattern: ^(https|s3)://([^/]+)/?(.*)$
model = Model(
    image_uri='135544376709.dkr.ecr.eu-west-1.amazonaws.com/mlops-pycaret-repo:latest',  # The ECR image you pushed
    model_data=model_data,  # Location of your serialized model
    role=role
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',
    endpoint_name=endpoint_name
)

# Clean up stage
## Remove Endpoint and Endpoint Config

In [ ]:
# Create a low-level SageMaker service client.
my_region = boto3.session.Session().region_name
sagemaker_client = boto3.client('sagemaker', region_name=my_region)

# Delete endpoint
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)

# Delete endpoint configuration
sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_name)